In [1]:
import pandas as pd
from rdkit import RDLogger,Chem                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')
df = pd.read_csv("mers_align_data/rcsb_pdb_ligand_20250312062454.csv")
pdb2ligands = {}
pdb2smis = {}
for i,eid in enumerate(df["Entity ID"]):
    if type(eid) == str: #ugly but it works
        current_eid = eid[:-2].lower()
        bestmw = 0
    lid = df["Ligand ID"][i]
    if type(lid) == str:
        mw = df["Ligand MW"][i]
        if mw > bestmw:
            pdb2ligands[current_eid] = df["Ligand ID"][i]
            pdb2smis[current_eid] = df["Ligand SMILES"][i]
            bestmw = mw
pdb2smis["mersA"] = "Clc1c(C2CC3(C(=O)N(c4c5c(cnc4)cccc5)C(=O)N3)C2)cccc1"

note that the below aligner fails for quite a few molecules. But it has all the ones i need in there so I dddn't toubleshoot

In [3]:
import biotite.structure as struc
import biotite.structure.io.pdb as pdb
from biotite.structure import superimpose_homologs
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.MolStandardize import rdMolStandardize
import tempfile
import os

DATA_FOLDER = "mers_align_data"

targets = [x[:-4] for x in os.listdir(DATA_FOLDER) if x[-3:]=="pdb"]
ref_pdb = pdb.PDBFile.read("mers_align_data/mersA.pdb")
ref_atoms = ref_pdb.get_structure()[0]
mols = [m for m in Chem.SDMolSupplier("ref_structs/MERS-CoV-Mpro/ligand.sdf")]
for t in targets:
    try:
        complex_pdb = pdb.PDBFile.read(f"mers_align_data/{t}.pdb")
        complex_atoms = complex_pdb.get_structure()[0]
        amino_acid_residues = [
            "ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
            "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"
        ]
        
        # Identify ligand atoms (non-standard residues)
        is_ligand_complex = complex_atoms.chain_id == "HET A"
        is_ligand_ref = ref_atoms.chain_id == "HET A"
        
        # Identify atoms belonging to Chain A
        is_chain_a_complex = complex_atoms.chain_id == "A"
        is_chain_a_ref = ref_atoms.chain_id == "A"
                
        # Keep Chain A **OR** ligands
        complex_atoms = complex_atoms[is_chain_a_complex | is_ligand_complex]
        ref_atoms = ref_atoms[is_chain_a_ref | is_ligand_ref]
                
        fitted,_,_,_ = superimpose_homologs(ref_atoms, complex_atoms)
        is_ligand = ~np.isin(fitted.res_name, amino_acid_residues+['HOH'])
        ligand_atoms = fitted[is_ligand]
        
        
        # the below roundabout procedure is needed in order to not have problems with bond orders etc
        # Create a temporary PDB file
        with tempfile.NamedTemporaryFile(suffix=".pdb", delete=False) as temp_pdb:
            out_pdb = pdb.PDBFile()
            out_pdb.set_structure(ligand_atoms)
            out_pdb.write(temp_pdb.name)
            temp_pdb_path = temp_pdb.name  # Store filename
        
        # Convert PDB to RDKit Mol object
        tmp = Chem.MolFromPDBFile(temp_pdb_path)
        largest_fragment = rdMolStandardize.LargestFragmentChooser()
        tmp = largest_fragment.choose(tmp)
        #this is the SDF provided by PDB for 6e9w
        template = Chem.MolFromSmiles(pdb2smis[t])
        
        #this is the aligned molecule you can use for calculating RMSDs
        mol = AllChem.AssignBondOrdersFromTemplate(template, tmp)
        mols.append(mol)
    except Exception as e:
        pass

wri = Chem.SDWriter("mersmols.sdf")
for m in mols:
    wri.write(m)
wri.close()